In [ ]:
import sys, os
import pandas as pd
import netCDF4 as nc
import numpy as np
sys.path.append('../src/')
from Biologging_Toolkit.applications.Mixed_Layer_Depth import MixedLayerDepth
from Biologging_Toolkit.processing.Dives import Dives
from Biologging_Toolkit.utils.format_utils import get_start_time_sens

In [ ]:
depid = 'ml18_294b'
path = os.path.join('/home/datawork-osmose/dataset/SES', depid)
ref_path = os.path.join(path, 'data', 'auxiliary', 'instrument')
sens_path = os.path.join(ref_path, depid+'sens5.nc')

### Make sure csv structure for dive data exists

In [ ]:
dive = Dives(depid, path = ref_path, sens_path = sens_path)

In [ ]:
dive()

### Add temperature data to reference structure

In [ ]:
ds = nc.Dataset(sens_path)
temperature = ds['T'][:].data
temp_time = get_start_time_sens(ds.dephist_device_datetime_start) + np.arange(0, len(temperature))/5

In [ ]:
dive.create_variable('temperature',
                     var_data =  temperature,
                     var_time = temp_time)

In [ ]:
dive.ds

In [ ]:
dive.ds.close()

### Compute mixed layer depth

In [ ]:
inst = MixedLayerDepth(depid, 
            path = ref_path
           )

In [ ]:
inst()

In [ ]:
from Biologging_Toolkit.utils.format_utils import *
up, down = inst.get_dive_direction(inst.depth[:: 60 // inst.ds.sampling_rate])
up, down = resample_boolean_array(up, len(inst.depth)), resample_boolean_array(down, len(inst.depth))
for dive in list(range(947,953)) :
    _zn1, _zn2 = [], []
    # Get estimations from two profiles : up and down trajectories
    for direction in [up, down]:
        _temp = inst.temp[(inst.dives == dive) & direction]
        _depth = inst.depth[(inst.dives == dive) & direction]
        #Sort temp and depth profile by increasing depth and check if there is enough data
        _depth, _temp = inst.profile_check(_depth, _temp, inst.ds.sampling_rate)
        delta_k, std_k, X_k = [],[],[]
        for k in range(len(_temp)):
            delta_k.append(np.sqrt( 1/(k+1) * np.nansum([((_temp[i] - np.nanmean(_temp[:k+1]))**2) for i in range(k)])))
            std_k.append(np.nanmax(_temp[:k+1]) - np.nanmin(_temp[:k+1]))
            X_k.append(delta_k[-1] / std_k[-1])
        #If X_k contains only nan values, add nan
        try : 
            pos_zn1 = np.nanmin((np.nanargmin(X_k), len(X_k)-2))  #If lowest X_k is last element, take penultimate element
            _zn1.append(_depth[pos_zn1])
        except ValueError :
            _zn1.append(np.nan)
            _zn2.append(np.nan)
            continue
        for step in range(len(delta_k[:pos_zn1 - 1]), -1, -1):
            if (delta_k[step + 1] - delta_k[step]) / (delta_k[pos_zn1 + 1] - delta_k[pos_zn1]) <= 0.3:
                _zn2.append(_depth[step])
                break